In [46]:
import pandas as pd
import numpy as np
import matplotlib as mp
import lightgbm as lgb
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as st
from scipy.stats import probplot, ks_2samp
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import (roc_auc_score, recall_score, accuracy_score, auc, classification_report, \
                            confusion_matrix, plot_confusion_matrix, precision_recall_curve, \
                            plot_precision_recall_curve, roc_curve, plot_roc_curve)

In [7]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import (roc_auc_score, roc_curve, auc, accuracy_score, confusion_matrix, \
                             classification_report, plot_confusion_matrix, recall_score, \
                             plot_precision_recall_curve, precision_recall_curve, plot_roc_curve)

In [72]:
import catboost as cb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [3]:
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
test = pd.read_csv('C:/Users/Anatoly/Desktop/Тестовое задание LICARD DS/DATA/contest_test.csv')

In [5]:
train = pd.read_csv('C:/Users/Anatoly/Desktop/Тестовое задание LICARD DS/DATA/contest_train.csv')

In [12]:
def clean_input(data) -> pd.DataFrame:
    """
    Перевод названий колонок в нижний регистр и вывод основной информации о наборе данных

    Parameters
    ----------
    data: Pandas DataFrame

    Returns
    -------
    data: pandas.core.frame.DataFrame
        Загруженный набор данных в pandas.DataFrame

    """
    #base_path = "data"
    #data = pd.read_csv(f"{base_path}/{data_path}")
    # Приводим все названия колонок к нижнему регистру
    data.columns = [col.lower() for col in data.columns]
    #print(f"{data_path}: shape = {data.shape[0]} rows, {data.shape[1]} cols")
    print(f"shape = {data.shape[0]} rows, {data.shape[1]} cols")

    return data

In [13]:
train = clean_input(train)
test = clean_input(test)

shape = 18390 rows, 262 cols
shape = 6131 rows, 261 cols


## Удаление лишних столбцов

In [15]:
numerical_features = train.select_dtypes(include=[np.number])

In [23]:
discrete_feature = [
    feature for feature in numerical_features
    if len(train[feature].unique())<25 
]

print(f"Discrete Variables Count: {len(discrete_feature)}")

Discrete Variables Count: 50


In [28]:
continuous_feature = [
    feature for feature in numerical_features
    if feature not in discrete_feature + ["ID"]]

print(f"Continuous Feature Count {len(continuous_feature)}")

Continuous Feature Count 212


In [29]:
train_con = train[continuous_feature]

In [30]:
test_con = test[continuous_feature]

In [24]:
train_descr = train[discrete_feature]

In [31]:
test_descr = test[continuous_feature]

In [16]:
feature_0 = []

for feat in numerical_features: 
    if (len(train[feat].unique()) == 1) and (len(test[feat].unique()) == 1) and (train[feat].unique() == 0): 
        feature_0.append(feat)

In [17]:
feature_0

['feature_3', 'feature_144', 'feature_249', 'feature_256']

In [20]:
# Также отберем и впоследствии удалим те бинарные фичи (0-1), в которых отношение нулевых признаков к единичным более 90%

In [18]:
threshold = train.shape[0]*0.9

In [19]:
threshold

16551.0

In [21]:
feature_90 = []

In [25]:
for feat in discrete_feature:
    try:
        if train_descr[feat].value_counts()[0] > threshold: 
            feature_90.append(feat) 
    except KeyError as ke: 
        continue

In [26]:
feature_90 = list(set(feature_90))

In [27]:
feature_90 

['feature_249',
 'feature_157',
 'feature_144',
 'feature_156',
 'feature_20',
 'feature_256',
 'feature_3',
 'feature_140',
 'feature_5',
 'feature_6',
 'feature_2',
 'feature_31',
 'feature_159',
 'feature_32']

In [32]:
for feat in continuous_feature:
    try:
        if train_con[feat].value_counts()[0] > threshold: 
            feature_90.append(feat) 
    except KeyError as ke: 
        continue

In [33]:
feature_90 

['feature_249',
 'feature_157',
 'feature_144',
 'feature_156',
 'feature_20',
 'feature_256',
 'feature_3',
 'feature_140',
 'feature_5',
 'feature_6',
 'feature_2',
 'feature_31',
 'feature_159',
 'feature_32',
 'feature_22',
 'feature_25',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_75',
 'feature_139',
 'feature_141',
 'feature_145',
 'feature_146',
 'feature_176',
 'feature_200',
 'feature_201',
 'feature_202',
 'feature_229',
 'feature_231',
 'feature_246']

In [34]:
similar_feature = [#"feature_209",
                  "feature_210", "feature_211", 
                  "feature_212", "feature_213", 
                  "feature_214"]
#or to delete?
# какой-то один признак нужно обязательно оставить. Например "feature_209"

In [35]:
train = train.drop(feature_90, axis=1)
test = test.drop(feature_90, axis=1)

In [36]:
train = train.drop(similar_feature, axis=1)
test = test.drop(similar_feature, axis=1)

In [37]:
train.shape, test.shape

((18390, 219), (6131, 218))

In [38]:
target = train['target']

In [39]:
feat_to_drop = ['id', 'target']

In [42]:
train = train.drop(feat_to_drop, axis=1)

In [43]:
test = test.drop('id', axis=1)

In [44]:
train.shape, test.shape

((18390, 217), (6131, 217))

In [57]:
data = train

## Validation_method_1

In [60]:
x_train, x_valid = train_test_split(
    data, train_size=0.5, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    target, train_size=0.5, shuffle=True, random_state=1,
)


x_valid, x_test = train_test_split(
    x_valid, train_size=0.7, shuffle=True, random_state=42
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.7, shuffle=True, random_state=42
)

In [61]:
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 9195 rows, 217 cols
x_valid.shape = 6436 rows, 217 cols
x_test.shape = 2759 rows, 217 cols


In [89]:
cb_params = {
    "n_estimators": 500,
    "loss_function": "MultiClass",
    "eval_metric": "TotalF1",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 50,
    "max_depth": 5,
    "l2_leaf_reg": 50,
    "early_stopping_rounds": 25,
    "thread_count": -1,
    "random_seed": 25
}

In [90]:
train_pool = cb.Pool(x_train, y_train)
valid_pool = cb.Pool(x_valid, y_valid)

In [91]:
model_cb = cb.CatBoostClassifier(**cb_params)
model_cb.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.5889538	test: 0.5847785	best: 0.5847785 (0)	total: 43ms	remaining: 21.4s
Stopped by overfitting detector  (25 iterations wait)

bestTest = 0.5847784821
bestIteration = 0

Shrink model to first 1 iterations.


## Crossvalidation

In [54]:
def lightgbm_cross_validation(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели LightGBM.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
#     if not categorical:
#         categorical = "auto"

    estimators, encoders = [], {}
    if categorical:
        for feature in categorical:
            encoder = LabelEncoder()
            X[feature] = encoder.fit_transform(X[feature].astype("str").fillna("NA"))
            encoders[feature] = encoder
        
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],
            eval_metric="multi_logloss", verbose=0, early_stopping_rounds=5000,
            #categorical_feature=categorical
        )
        oof_preds[valid_idx] = model.predict(x_valid)[:, 1]
        oof_preds[valid_idx] = [np.argmax(x) for x in oof_preds[valid_idx]]
        score = f1_score(y_valid, oof_preds[valid_idx], average='macro')
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [55]:
lgbm_params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'n_estimators': 10000,
    'learning_rate': 0.05134,
    'num_leaves': 54,
    'max_depth': 10,
    'num_class': 3,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 0.7,
    'is_unbalance': False,
    'random_state': 27,
    'silent': -1,
    'verbose': -1,
}

In [56]:
cv = KFold(n_splits=8, random_state=282, shuffle=True)

lgb_estimators, lgb_oof_preds = lightgbm_cross_validation(
    params=lgbm_params, X=train, y=target, cv=cv)

# метрика качества
lgb_oof_score = f1_score(
    target, lgb_oof_preds
)
print(f"OOF-score = {lgb_oof_score}")

Wed Mar 16 18:19:40 2022, Cross-Validation, 18390 rows, 217 cols


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed